In [1]:
import requests
from lxml import html

import pandas as pd

import uuid

# request, bypass certificate check
req = requests.get('https://ceb.lk', verify=False)
webpage = html.fromstring(req.content)
links = webpage.xpath('//a/@href')
# look for google drive link
gd_link = [i for i in links if 'drive.google.com' in i]

# remove duplicates
gd_link = list(set(gd_link))

print(gd_link)

/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ceb.lk'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


['https://drive.google.com/file/d/1GcHsPqBYP9oT-EAetHyx1zgDGdXHcLNu/view?usp=sharing']


In [2]:
import requests
import camelot

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

if __name__ == "__main__":
    file_id = gd_link[0].split('/')[5]
    destination = './assets/ceb_googledoc.pdf'
    download_file_from_google_drive(file_id, destination)

In [3]:
# read tables
tables = camelot.read_pdf('./assets/ceb_googledoc.pdf')

In [8]:
from datetime import datetime
import pytz
  
timeZ_colomo = pytz.timezone('Asia/Colombo')

datetime.now(timeZ_colomo).strftime('%Y-%m-%d')

'2022-04-09'

In [27]:
sl_time = datetime.now(pytz.timezone('Asia/Colombo')).strftime('%Y-%m-%d')

def convert_time(time_str, time_date = sl_time):
    time_str = time_date+'T'+time_str+':00.000Z'   
    return time_str

def process_tables(tables):
    
    dff = pd.DataFrame()
    
    for ii in range(2):
        
        df = tables[ii].df
        
        df = pd.DataFrame(data = df.iloc[1:].values, columns = df.iloc[0])
        df['start_time'] = df['Period'].apply(lambda x: x.split(' ')[0] if ' ' in x else x.split('-')[0])
        df['end_time'] = df['Period'].apply(lambda x: x.split(' ')[-1] if ' ' in x else x.split('-')[-1])
        
        df['start_time'] = df['start_time'].apply(lambda x: x.replace('–','')).apply(lambda x: convert_time(x))
        df['end_time'] = df['end_time'].apply(lambda x: x.replace('–','')).apply(lambda x: convert_time(x))
        
        for jj in range(len(df)):
            eles = df['Schedule Group'][jj].replace(' ','').split(',')
            eles = ' '.join(eles).split()
            for ele in eles:
                dff = dff.append(pd.DataFrame(data = {'group_name':ele,'starting_period':[df.iloc[jj]['start_time']],
                                                  'ending_period':[df.iloc[jj]['end_time']],'unique_id': str(uuid.uuid1())}))
    
    
    return dff
    

dff = process_tables(tables).reset_index(drop = True)

dff.head()

,group_name,starting_period,ending_period,unique_id
0,A,2022-04-09T15:00:00.000Z,2022-04-09T17:00:00.000Z,d6da6d26-b828-11ec-a3a7-8c85903cbddd
1,B,2022-04-09T15:00:00.000Z,2022-04-09T17:00:00.000Z,d6daa64c-b828-11ec-a3a7-8c85903cbddd
2,C,2022-04-09T15:00:00.000Z,2022-04-09T17:00:00.000Z,d6dad8e2-b828-11ec-a3a7-8c85903cbddd
3,D,2022-04-09T17:00:00.000Z,2022-04-09T19:00:00.000Z,d6db0416-b828-11ec-a3a7-8c85903cbddd
4,E,2022-04-09T17:00:00.000Z,2022-04-09T19:00:00.000Z,d6db3526-b828-11ec-a3a7-8c85903cbddd


In [28]:
json_out = process_tables(tables).reset_index(drop = True).to_json(orient = 'records')

json_out

'[{"group_name":"A","starting_period":"2022-04-09T15:00:00.000Z","ending_period":"2022-04-09T17:00:00.000Z","unique_id":"d7afead2-b828-11ec-a3a7-8c85903cbddd"},{"group_name":"B","starting_period":"2022-04-09T15:00:00.000Z","ending_period":"2022-04-09T17:00:00.000Z","unique_id":"d7b00c74-b828-11ec-a3a7-8c85903cbddd"},{"group_name":"C","starting_period":"2022-04-09T15:00:00.000Z","ending_period":"2022-04-09T17:00:00.000Z","unique_id":"d7b0380c-b828-11ec-a3a7-8c85903cbddd"},{"group_name":"D","starting_period":"2022-04-09T17:00:00.000Z","ending_period":"2022-04-09T19:00:00.000Z","unique_id":"d7b065a2-b828-11ec-a3a7-8c85903cbddd"},{"group_name":"E","starting_period":"2022-04-09T17:00:00.000Z","ending_period":"2022-04-09T19:00:00.000Z","unique_id":"d7b08f32-b828-11ec-a3a7-8c85903cbddd"},{"group_name":"F","starting_period":"2022-04-09T17:00:00.000Z","ending_period":"2022-04-09T19:00:00.000Z","unique_id":"d7b0b9c6-b828-11ec-a3a7-8c85903cbddd"},{"group_name":"G","starting_period":"2022-04-09T18

In [5]:
# DOES NOT WORK

# import PyPDF2

# # creating a pdf file object
# pdfFileObj = open('ceb_googledoc.pdf', 'rb')
 
# # creating a pdf reader object
# pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

# # creating a page object
# pageObj = pdfReader.getPage(0)
 
# # extracting text from page
# print(pageObj.extractText())
 
# # closing the pdf file object
# pdfFileObj.close()

In [191]:
json_out

'[{"group":"A","start_time":"15:00","end_time":"17:00"},{"group":"B","start_time":"15:00","end_time":"17:00"},{"group":"C","start_time":"15:00","end_time":"17:00"},{"group":"D","start_time":"17:00","end_time":"19:00"},{"group":"E","start_time":"17:00","end_time":"19:00"},{"group":"F","start_time":"17:00","end_time":"19:00"},{"group":"G","start_time":"18:00","end_time":"20:00"},{"group":"H","start_time":"18:00","end_time":"20:00"},{"group":"I","start_time":"18:00","end_time":"20:00"},{"group":"J","start_time":"19:00","end_time":"21:00"},{"group":"K","start_time":"19:00","end_time":"21:00"},{"group":"L","start_time":"19:00","end_time":"21:00"},{"group":"P","start_time":"15:00","end_time":"17:00"},{"group":"Q","start_time":"15:00","end_time":"17:00"},{"group":"R","start_time":"17:00","end_time":"19:00"},{"group":"S","start_time":"17:00","end_time":"19:00"},{"group":"T","start_time":"18:00","end_time":"20:00"},{"group":"U","start_time":"18:00","end_time":"20:00"},{"group":"V","start_time":

In [185]:
# process_tables(tables).reset_index(drop = True).head(20)

,Group,start_time,end_time
0,A,15:00,17:00
1,B,15:00,17:00
2,C,15:00,17:00
3,D,17:00,19:00
4,E,17:00,19:00
5,F,17:00,19:00
6,G,18:00,20:00
7,H,18:00,20:00
8,I,18:00,20:00
9,J,19:00,21:00


In [169]:
df_tmp = tables[0].df

df_tmp = pd.DataFrame(data = df_tmp.iloc[1:].values, columns = df_tmp.iloc[0])

df_tmp['start_time'] = df_tmp['Period'].apply(lambda x: x.split(' ')[0] if ' ' in x else x.split('-')[0])
df_tmp['end_time'] = df_tmp['Period'].apply(lambda x: x.split(' ')[0] if ' ' in x else x.split('-')[-1])

test_list = ' '.join(test_list).split()

df_tmp['Schedule Group'].iloc[2].replace(' ','').split(',')

['G', 'H', 'I', '']

In [170]:
df_tmp

,Period,Schedule Group,start_time,end_time
0,15:00 – 17:00,"A, B, C",15:00,15:00
1,17:00 –19:00,"D, E, F",17:00,17:00
2,18:00 – 20:00,"G, H, I,",18:00,18:00
3,19:00 – 21:00,"J, K, L",19:00,19:00


In [150]:
process_tables(tables).reset_index(drop = True).to_json()

'{"Group":{"0":"A","1":"B","2":"C","3":"D","4":"E","5":"F","6":"G","7":"H","8":"I","9":"","10":"J","11":"K","12":"L","13":"P","14":"Q","15":"R","16":"S","17":"T","18":"U","19":"","20":"V","21":"W"},"start_time":{"0":"15:00","1":"15:00","2":"15:00","3":"17:00","4":"17:00","5":"17:00","6":"18:00","7":"18:00","8":"18:00","9":"18:00","10":"19:00","11":"19:00","12":"19:00","13":"15:00","14":"15:00","15":"17:00","16":"17:00","17":"18:00","18":"18:00","19":"18:00","20":"20:00","21":"20:00"},"end_time":{"0":"15:00","1":"15:00","2":"15:00","3":"17:00","4":"17:00","5":"17:00","6":"18:00","7":"18:00","8":"18:00","9":"18:00","10":"19:00","11":"19:00","12":"19:00","13":"17:00","14":"17:00","15":"19:00","16":"19:00","17":"20:00","18":"20:00","19":"20:00","20":"22:00","21":"22:00"}}'

In [137]:
df0 = tables[0].df

def process_df(df):
    df = pd.DataFrame(data = df.iloc[1:].values, columns = df.iloc[0])
    df['start_time'] = df['Period'].apply(lambda x: x.split(' ')[0]).str.replace('–','')
    df['end_time'] = df['Period'].apply(lambda x: x.split(' ')[-1]).str.replace('–','')
    
    dff = pd.DataFrame(columns = ['start_time', 'end_time'])
    
    for ii in range(len(df)):
        eles = df['Schedule Group'][ii].replace(' ','').split(',')
        for ele in eles:
            dff = dff.append(pd.DataFrame(data = {'start_time':[df.iloc[ii]['start_time']],
                                                  'end_time':[df.iloc[ii]['end_time']]}, index = [ele]))
    
    return dff

dff0 = process_df(tables[2].df)

In [148]:
df = tables[2].df

df = pd.DataFrame(data = df.iloc[1:].values, columns = df.iloc[0])
df['start_time'] = df['Period'].apply(lambda x: x.split(' ')[0] if ' ' in x else x.split('-')[0])
df['end_time'] = df['Period'].apply(lambda x: x.split(' ')[0] if ' ' in x else x.split('-')[-1])

df

,Period,Schedule Group,start_time,end_time
0,06:00-9:30,CC1,06:00,9:30


In [136]:
tables[1].df

,0,1
0,Period,Schedule Group
1,15:00-17:00,"P, Q"
2,17:00-19:00,"R, S"
3,18:00-20:00,"T, U,"
4,20:00-22:00,"V, W"


In [4]:
df0 = tables[0].df

df0.head()

,0,1
0,Period,Schedule Group
1,15:00 – 17:00,"A, B, C"
2,17:00 –19:00,"D, E, F"
3,18:00 – 20:00,"G, H, I,"
4,19:00 – 21:00,"J, K, L"


In [18]:
import pandas as pd
df0 = pd.DataFrame(data = df0.iloc[1:].values, columns = df0.iloc[0])
df0.head()

,Period,Schedule Group
0,15:00 – 17:00,"A, B, C"
1,17:00 –19:00,"D, E, F"
2,18:00 – 20:00,"G, H, I,"
3,19:00 – 21:00,"J, K, L"


In [35]:
df0.iloc[0]['Period'].split(' ')[0]

'15:00'

In [40]:
df0['start_time'] = df0['Period'].apply(lambda x: x.split(' ')[0])

0    15:00
1    17:00
2    18:00
3    19:00
Name: Period, dtype: object

In [30]:
data = {'A':{'start_time': '15:00', 'end_time':'17:00', 'GSS':''}}

data

{'A': {'start_time': '15:00', 'end_time': '17:00', 'GSS': ''}}

In [14]:
tables[1].df

,0,1
0,Period,Schedule Group
1,15:00-17:00,"P, Q"
2,17:00-19:00,"R, S"
3,18:00-20:00,"T, U,"
4,20:00-22:00,"V, W"


In [6]:
tables[2].df

,0,1
0,Period,Schedule Group
1,06:00-9:30,CC1


In [23]:
tables[3].df

,0,1,2
0,GSS,Feeder No,Affected area
1,Ambalangoda,1,"Agaliya, Kahaduwa,"
2,Athurugiriya,1,"Mulleriyawa North, Welivita,Kaduwela, Hewagama..."
3,Balangoda,2,"Weligopolatown,Kongasthanna,Yatipasgamuwa,Kott..."
4,Deniyaya,1,"Lions Tea Factory,Puhulahenakanda,,Ketawala,Th..."
5,Embilipitiya,8,"Middeniya, Panamura, Mullendiyawala, Jandura, ..."
6,Horana,2,Horana Hospital
7,Kalutara,6,"waterboard (Kethhena) ,kiwlawatta ,Mankada,Deh..."
8,Kiribathkumbura,11,Kandy City Supply
9,Maliboda,2,"Udahenkanda, Samanpuragama, Deraniyagala Town,..."


In [7]:
# Copyright 2019 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""
Recursively extracts the text from a Google Doc.
"""
from __future__ import print_function

import googleapiclient.discovery as discovery
from httplib2 import Http
from oauth2client import client
from oauth2client import file
from oauth2client import tools

SCOPES = 'https://www.googleapis.com/auth/documents.readonly'
DISCOVERY_DOC = 'https://docs.googleapis.com/$discovery/rest?version=v1'
DOCUMENT_ID = 'YOUR_DOCUMENT_ID'


def get_credentials():
    """Gets valid user credentials from storage.

    If nothing has been stored, or if the stored credentials are invalid,
    the OAuth 2.0 flow is completed to obtain the new credentials.

    Returns:
        Credentials, the obtained credential.
    """
    store = file.Storage('token.json')
    credentials = store.get()

    if not credentials or credentials.invalid:
        flow = client.flow_from_clientsecrets('credentials.json', SCOPES)
        credentials = tools.run_flow(flow, store)
    return credentials

def read_paragraph_element(element):
    """Returns the text in the given ParagraphElement.

        Args:
            element: a ParagraphElement from a Google Doc.
    """
    text_run = element.get('textRun')
    if not text_run:
        return ''
    return text_run.get('content')


def read_structural_elements(elements):
    """Recurses through a list of Structural Elements to read a document's text where text may be
        in nested elements.

        Args:
            elements: a list of Structural Elements.
    """
    text = ''
    for value in elements:
        if 'paragraph' in value:
            elements = value.get('paragraph').get('elements')
            for elem in elements:
                text += read_paragraph_element(elem)
        elif 'table' in value:
            # The text in table cells are in nested Structural Elements and tables may be
            # nested.
            table = value.get('table')
            for row in table.get('tableRows'):
                cells = row.get('tableCells')
                for cell in cells:
                    text += read_structural_elements(cell.get('content'))
        elif 'tableOfContents' in value:
            # The text in the TOC is also in a Structural Element.
            toc = value.get('tableOfContents')
            text += read_structural_elements(toc.get('content'))
    return text


def main():
    """Uses the Docs API to print out the text of a document."""
    credentials = get_credentials()
    http = credentials.authorize(Http())
    docs_service = discovery.build(
        'docs', 'v1', http=http, discoveryServiceUrl=DISCOVERY_DOC)
    doc = docs_service.documents().get(documentId=DOCUMENT_ID).execute()
    doc_content = doc.get('body').get('content')
    print(read_structural_elements(doc_content))

if __name__ == '__main__':
    main()

/opt/anaconda3/lib/python3.9/site-packages/oauth2client/_helpers.py:255: UserWarning: Cannot access token.json: No such file or directory
  warnings.warn(_MISSING_FILE_MESSAGE.format(filename))


InvalidClientSecretsError: Missing property "redirect_uris" in a client type of "web".